In [1]:
import numpy as np
import pandas as pd
import glob
import ipyparallel as p
import os
import ntpath
import ProgressBar as pb
from pymail import alert
import time

In [2]:
exname = 'smex7_test'
method = '7'
site = 'Nr1'

In [3]:
files = glob.glob('/Volumes/data/RHESSys_out/Nwt/'+exname+'/*_basin.daily')

In [4]:
params = pd.read_pickle('./data/params_%s_smex_method%s.pcl'%(site,method))

In [5]:
files = pd.DataFrame({'basin_daily':files})

In [6]:
def getIDX(fl):
    return int(fl.split('_')[-2])

In [7]:
files['idx'] = files.basin_daily.map(getIDX)

In [8]:
params = pd.merge(params,files, on= 'idx')

In [9]:
c = p.Client()
view = c.load_balanced_view()

In [10]:
c.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [11]:
len(params)

50000

In [12]:
%%px
import os

def wateryear(index):
    year = index.year
    strt = pd.datetime(int(year),10,1) # start of the next water year+1
    
    if index<strt:
        wyear = year
    elif index>=strt: 
        wyear = year+1

    return wyear

def readRHESSysBasin(filename):
        
    if os.path.isfile(filename) == False:
        print 'Input file does not exist'
    else:
        bd = filename # basin output path

        bddat = pd.read_table(bd,delim_whitespace=True,
                               header='infer', parse_dates={'foo':[2,1,0]}, index_col='foo')
        bddat.reset_index(inplace=True)
        #bddat.index = pd.DatetimeIndex(bddat['foo'])

        data = bddat
        del data['foo']
        
        #data['wateryear'] = data.index.map(wateryear) # calculate the water year
        
        # calculate accumulated precip and snowfall by water year
        
        data['ET'] = data.evap+data.trans
        
        return data

In [13]:
def readRHESSysBasin(filename):
        
    if os.path.isfile(filename) == False:
        print 'Input file does not exist'
    else:
        bd = filename # basin output path

        bddat = pd.read_table(bd,delim_whitespace=True,
                               header='infer', parse_dates={'foo':[2,1,0]}, index_col='foo')
        bddat.reset_index(inplace=True)
        #bddat.index = pd.DatetimeIndex(bddat['foo'])

        data = bddat
        del data['foo']
        
        #data['wateryear'] = data.index.map(wateryear) # calculate the water year
        
        # calculate accumulated precip and snowfall by water year
        
        data['ET'] = data.evap+data.trans
        
        return data

In [14]:
dr = pd.date_range('2009-10-1','2010-9-30',freq='D')

In [15]:
%px import os

In [50]:
#x = params.iloc[60000]

#bd = x.basin_daily
#patchd = x.patch_daily
#idx = x.idx
#strt = x.date_peakSWE
#nd = x.date_ONS

def processSM(bd,idx,strt,nd):
    tmp = np.NaN
    dr = pd.date_range('2009-10-1','2010-10-02',freq='D')
    
    lt = len(dr)

    # read in the basin daily data
    data = pd.read_table(bd, delim_whitespace=True)

    if len(data) < lt:
        return idx,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp
    else:
        data.index = pd.DatetimeIndex(dr)
        
        

        data = data.loc[strt:nd] # crop the data to the melt season

        data['ET'] = data.evap + data.trans

        ET = data.ET.sum() # compute total ET
        Q = data.streamflow.sum() # compute total streamflow 
        Qbf = data.baseflow.sum() # compute total baseflow
        Qrf = data['return'].sum() # compute total returnflow,p
        sm = data.snowmelt.sum() # compute snowmelt from model
        rech = data.recharge.sum() # compute total recharge
        sat = data['%sat_area'].mean() # compute the mean saturated area
        p = data.precip.sum() # compute water year precip
        rz_storage = data.rz_storage.mean() # root zone storage
        rz_drainage = data.rz_drainage.mean() # root zone drainage 
        unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
        unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
        sat_def = data.sat_def.mean() # saturation deficite (mm)
        sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?

    # read in the patch data

    #data = pd.read_table(patchd, delim_whitespace=True) # load the patch data
    
    #if len(data) < lt:
    #    return idx,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp,tmp
    #else:
    #    data.index = pd.DatetimeIndex(dr)
    #    data = data.loc[strt:nd] # crop the data to the melt season
#
#        Qin = data.Qin.mean() # saturated flow in
#        Qout = data.Qout.mean() # saturated flow out
#        streamflow = data.streamflow.mean() # streamflow
#        rz_field_capacity = data.rz_field_capacity.mean() # mean field capacity
#        snow_melt = data.snow_melt.sum()
    
        return idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm
    #,Qin,Qout,streamflow,rz_field_capacity,snow_melt


#print idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,idx,Qin,Qout,streamflow,rz_field_capacity,snow_melt

In [16]:
%px dr = pd.date_range('2009-10-1','2010-9-30',freq='D')

In [17]:
def processSM(bd,idx,strt,nd):

    # read in the basin daily data
    data = pd.read_table(bd,delimiter=' ')

    data.index = pd.DatetimeIndex(dr)

    data['ET'] = data.evap + data.trans

    ET = data.ET.sum() # compute total ET
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    # create full output object
    full = (idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    data = data.loc[strt:nd] # crop the data to the melt season
    
    ET = data.ET.sum() # compute total ET
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    melt = (idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    return full,melt

In [18]:
idx = 2
print params.basin_daily[idx]
processSM(params.basin_daily[idx],params.idx[idx],params.date_peakSWE[idx],params.date_ONS[idx])

/Volumes/data/RHESSys_out/Nwt/smex7_test/smex7_test_2_basin.daily


((2,
  449.7278710000001,
  207.78034800000015,
  207.78034800000015,
  0.0,
  252.9876059999999,
  0.0,
  662.319009,
  1541.4915468904094,
  0.0,
  1165.197779999992,
  0.0,
  5478.913440000021,
  11441.0,
  254.863824,
  3785.8371879999995,
  0.5692612273972607,
  1.2321311534246577),
 (2,
  17.496865,
  7.0646759999999995,
  7.0646759999999995,
  0.0,
  53.065095,
  0.0,
  0.0,
  1547.9127248421053,
  0.0,
  1165.19778,
  0.0,
  5478.913440000002,
  11441.0,
  96.657731,
  165.247478,
  0.37182505263157895,
  0.9208876315789474))

In [27]:
#res = view.map(processSM,params.basin_daily[0:1000],params.idx[0:1000],params.date_peakSWE[0:1000],params.date_ONS[0:1000])
res = view.map(processSM,params.basin_daily,params.idx,params.date_peakSWE,params.date_ONS)

In [28]:
prog = pb.ProgressBar(len(params))

In [29]:
while res.ready() == False:
    prog.animate_ipython(res.progress)
    time.sleep(2)

alert.send_alert('barnhatb@colorado.edu','Processing Nr1 %s has finished'%exname,'Your script has finished')

[*****************99%******************]  49464 of 50000 complete


In [30]:
res.done()

True

In [31]:
res.progress

50000

In [32]:
full,melt = zip(*res.result())

## Merge processing results with parameters dataset

In [33]:
idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*full)

params = pd.read_pickle('./data/params_%s_smex_method%s.pcl'%(site,method)) # load parameters dataset

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['idx','ET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='idx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_long_full_test.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True)

In [34]:
idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*melt)

params = pd.read_pickle('./data/params_%s_smex_method%s.pcl'%(site,method)) # load parameters dataset

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['idx','ET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='idx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_long_melt_test.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True)